In [1]:
# default_exp data

# data

> Read data from netdata rest api into a pandas dataframe.

In [2]:
# hide
from nbdev.showdoc import *

In [3]:
# hide
# export
import asks
from asks import BasicAuth
import trio
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from netdata_pandas.wrangle import drop_low_uniqueness_cols, drop_low_std_cols

In [4]:
# export


def get_chart_list(host: str = '127.0.0.1:19999', starts_with: str = None) -> list:
    """Get list of all available charts on a `host`.  
    
    ##### Parameters:  
    - **host** `str` The host we want to get a list of available charts from.
    - **starts_with** `str` A string to filter the list of charts returns to just those that start with `starts_with`.
    
    ##### Returns:  
    - **chart_list** `list` A list of availalbe charts.
    
    """    
    url = f"http://{host}/api/v1/charts"
    r = requests.get(url)
    charts = r.json().get('charts')
    chart_list = [chart for chart in charts]
    if starts_with:
        chart_list = [chart for chart in chart_list if chart.startswith(starts_with)]
    return chart_list



In [5]:
# hide
# tests

# get some charts from london demo site
charts = get_chart_list('london.my-netdata.io', starts_with='system.')

# check just system. charts returned
assert set([chart.split('.')[0] for chart in charts]) == set(['system'])


In [6]:
# export


async def get_chart(api_call: str, data: list, col_sep: str ='|', numeric_only: bool = True, float_size: str = 'float64'):
    """Get data for an individual chart.
    
    ##### Parameters:  
    - **api_call** `tuple` A tuple of (`url`,`chart`) for the url to pull data from and chart it represents.
    - **data** `list` A list for dataframes for each chart to be appended to.
    - **col_sep** `str` A character for separating chart and dimension in column names of dataframe.
    - **numeric_only** `bool` Set to true if you want to filter out any non numeric data.
    - **float_size** `str` float size to use if would like to save some memory, eg can use 'float32' or 'float16'.
    
    """
    url, chart, host, user, pwd = api_call
    if user and pwd:
        user_pwd = (user, pwd)
        r = await asks.get(url, auth=BasicAuth(user_pwd))
    else:
        r = await asks.get(url)
    r_json = r.json()
    df = pd.DataFrame(r_json['data'], columns=['time_idx'] + r_json['labels'][1:])
    df['host'] = host
    df = df.set_index(['host','time_idx']).add_prefix(f'{chart}{col_sep}')
    if numeric_only:
        df = df._get_numeric_data().astype(float_size)
    data.append(df)



In [7]:
# export


async def get_charts(api_calls: list, col_sep: str ='|', timeout: int = 60, numeric_only: bool = True, float_size: str = 'float64') -> pd.DataFrame:
    """Create a nursey to make seperate async calls to get each chart.
    
    ##### Parameters:  
    - **api_calls** `list` A list of tuple's of [(`url`,`chart`),...] of api calls that need to be made.
    - **col_sep** `str` A character for separating chart and dimension in column names of dataframe.
    - **timeout** `int` The number of seconds for trio to [move_on_after](https://trio.readthedocs.io/en/stable/reference-core.html#trio.move_on_after).
    - **numeric_only** `bool` Set to true if you want to filter out any non numeric data.
    - **float_size** `str` float size to use if would like to save some memory, eg can use 'float32' or 'float16'.
    
    ##### Returns:  
    - **df** `pd.DataFrame` A pandas dataframe with all chart data outer joined based on time index.
    
    """
    n_hosts = len(set([x[2] for x in api_calls]))
    data = []
    with trio.move_on_after(timeout):
        async with trio.open_nursery() as nursery:
            for api_call in api_calls:
                nursery.start_soon(get_chart, api_call, data, col_sep, numeric_only, float_size)
    if n_hosts == 1:
        df = pd.concat(data, join='outer', axis=1, sort=True)
    else:
        df = pd.concat(data, join='outer', axis=0, sort=True)
    return df



In [8]:
# export


def get_data(hosts: list = ['london.my-netdata.io'], charts: list = ['system.cpu'], after: int = -60, 
             before: int = 0, points: int = 0, col_sep: str = '|', numeric_only: bool = True,
             ffill: bool = True, diff: bool = False, timeout: int = 60, nunique_thold = None, 
             std_thold: float = None, index_as_datetime: bool = False, freq: str = 'infer', 
             group: str = 'average', sort_cols: bool = True, user: str = None, pwd: str = None, 
             protocol: str = 'http', sort_rows: bool = True, float_size: str = 'float64') -> pd.DataFrame:
    """Define api calls to make and any post processing to be done.
    
    ##### Parameters:  
    - **hosts** `list` A list of hosts to pull data from.
    - **charts** `list` A list of charts to pull data for.
    - **after** `int` The timestamp or relative integer from which to pull data after.
    - **before** `int` The timestamp or relative integer from which to pull data before.
    - **points** `int` The `points` parameter to pass to the api call if need to aggregate data in some way.
    - **col_sep** `str` A character for separating chart and dimension in column names of dataframe.
    - **numeric_only** `bool` Set to true if you want to filter out any non numeric data.
    - **ffill** `bool` Set to true if you want to forward fill any null or missing values.
    - **diff** `bool` Set to true if you want to get the difference of metrics as opposed to their raw value.
    - **timeout** `int` The number of seconds for trio to [move_on_after](https://trio.readthedocs.io/en/stable/reference-core.html#trio.move_on_after).
    - **nunique_thold** [`float`,`int`] If defined calls function to filter cols with low number of unique values.
    - **std_thold** `float` If defined calls function to filter cols with low standard deviation.
    - **index_as_datetime** `bool` If true, set the index to be a pandas datetime.
    - **freq** `str` Freq to be passed to pandas datetime index.
    - **group** `str` The grouping function to use in the netdata api call.
    - **sort_cols** `bool` True to sort columns by name.
    - **user** `str` A username to use if netdata is password protected.
    - **pwd** `str` A password to use if netdata is password protected.
    - **protocol** `str` 'http' or 'https'.
    - **sort_rows** `bool` True to sort rows by index.
    - **float_size** `str` float size to use if would like to save some memory, eg can use 'float32' or 'float16'.
        
    ##### Returns:  
    - **df** `pd.DataFrame` A pandas dataframe with all chart data outer joined based on time index and any post processing done.
    
    """
    # if hosts is a string make it a list of one
    if isinstance(hosts, str):
        hosts = [hosts]
    
    # get list of host chart tuples we need to get data for
    if charts == ['all']:
        host_charts = [(host, chart) for host in hosts for chart in get_chart_list(host)]
    else:
        host_charts = [(host, chart) for host in hosts for chart in charts]
    
    # define list of all api calls to be made
    api_calls = [
        (f'{protocol}://{host_chart[0]}/api/v1/data?chart={host_chart[1]}&after={after}&before={before}&points={points}&format=json&group={group}', host_chart[1], host_chart[0], user, pwd)
        for host_chart in host_charts
    ] 
    # get the data
    df = trio.run(get_charts, api_calls, col_sep, timeout, numeric_only, float_size)
    # post process the data
    df = df.groupby(by=['host','time_idx']).max()
    if len(hosts) == 1:
        df = df.reset_index(level=0, drop=True)
    if sort_rows:
        df = df.sort_index()
    if ffill:
        df = df.ffill()
    if diff:
        df = df.diff().dropna(how='all')
    if nunique_thold:
        df = drop_low_uniqueness_cols(df, nunique_thold)
    if std_thold:
        df = drop_low_std_cols(df, std_thold)
    if index_as_datetime:
        df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index, unit='s'), freq=freq))
    if sort_cols:
        df = df.reindex(sorted(df.columns), axis=1)
    return df



In [9]:
# hide
%timeit -r1 -n2 get_data('london.my-netdata.io', ['all'], after=-600, before=0)

5.57 s ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [10]:
# hide
%timeit -r1 -n2 get_data('london.my-netdata.io', ['system.cpu', 'system.load'], after=-600, before=0)

117 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [11]:
# hide
# tests
# examples

df = get_data('london.my-netdata.io', ['all'], after=-60, before=0, nunique_thold=0.05)
assert len(df) in [59, 60, 61, 62, 63, 64, 65, 66, 67]
assert len(df.columns) > 500
assert len(df.columns) < 1000
print(df.shape)
df.head()

(65, 559)


,apps.cpu_system|apps.plugin,apps.cpu_system|charts.d.plugin,apps.cpu_system|httpd,apps.cpu_system|kernel,apps.cpu_system|netdata,apps.cpu_system|python.d.plugin,apps.cpu_system|sql,apps.cpu_system|tc-qos-helper,apps.cpu_system|vpn,apps.cpu_user|apps.plugin,...,web_log_nginx.requests_per_ipproto|ipv6,web_log_nginx.requests_per_url|allmetrics,web_log_nginx.requests_per_url|charts,web_log_nginx.requests_per_url|kickstart,web_log_nginx.requests_per_url|other,web_log_nginx.requests_per_url|registry,web_log_nginx.response_codes|2xx,web_log_nginx.response_statuses|success,web_log_nginx.response_time|avg,web_log_nginx.response_time|max
time_idx,,,,,,,,,,,,,,,,,,,,,
1603455170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1603455171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,9.528711,0.000000,1.264898,2.132216,12.925825,12.925825,0.524307,0.999845
1603455172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,11.602048,0.867326,1.867334,1.132689,15.469398,15.469398,0.283172,1.000008
1603455173,0.8537,0.0,0.0,0.0,0.8537,0.0,0.8537,0.0,0.0,0.0,...,0.867339,0.0,7.663457,0.132674,2.000025,2.734691,12.530847,12.530847,0.611684,1.000013
1603455174,0.9999,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,...,0.132661,0.0,11.336895,0.000000,1.132678,0.397983,12.867556,12.867556,0.688695,1.867372


In [12]:
# hide
# tests

# get some test data
test_host = 'london.my-netdata.io'
test_charts = ['system.cpu', 'system.load']
df = get_data(test_host, test_charts, after=-60, before=0, col_sep='|')

# look for some expected columns
assert 'system.load|load1' in df.columns
assert 'system.cpu|user' in df.columns
# check expected shape of data
assert str(df.shape) == '(60, 12)' or '(61, 12)'
# check that all types are float64 or int64
assert len(df.dtypes[df.dtypes != 'int64'][df.dtypes != 'float64']) == 0

# test index as datetime
df = get_data('london.my-netdata.io', ['system.cpu'], index_as_datetime=True)
assert isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex)
assert isinstance(df.index.freq, pd.tseries.offsets.Second)

# test multiple hosts and charts
df = get_data(['london.my-netdata.io', 'newyork.my-netdata.io'], ['system.cpu', 'system.load'], after=-60, before=0)
assert df.shape[0] in [120, 121, 122]
assert df.shape[1] == 12
assert list(df.columns) == ['system.cpu|guest', 'system.cpu|guest_nice', 'system.cpu|iowait', 'system.cpu|irq', 'system.cpu|nice', 'system.cpu|softirq', 
                            'system.cpu|steal', 'system.cpu|system', 'system.cpu|user', 'system.load|load1', 'system.load|load15', 'system.load|load5']

# test memory savings from float 32
df64 = get_data('london.my-netdata.io', test_charts, after=-600, before=0)
df32 = get_data('london.my-netdata.io', test_charts, after=-600, before=0, float_size='float32')
assert df32.memory_usage('deep').sum() < df64.memory_usage('deep').sum()

In [13]:
# export


def get_alarm_log(host: str = '127.0.0.1:19999', datetimes: bool = True, user: str = None, 
                  pwd: str = None, protocol: str = 'http') -> pd.DataFrame:
    """Get alarm log from `host`.  
    
    ##### Parameters:  
    - **host** `str` The host we want to get the alarm log from.
    - **user** `str` A username to use if netdata is password protected.
    - **pwd** `str` A password to use if netdata is password protected.
    - **protocol** `str` 'http' or 'https'.
    
    ##### Returns:  
    - **df** `pd.DataFrame` A df of the alarm_log.
    
    """
    
    url = f"{protocol}://{host}/api/v1/alarm_log"
    if user and pwd:
        r = requests.get(url, auth=HTTPBasicAuth(user, pwd))
    else:
        r = requests.get(url)
    alarm_log = r.json()
    df = pd.DataFrame(alarm_log)
    if datetimes:
        for col in ['when', 'delay_up_to_timestamp']:
            df[col] = pd.to_datetime(df[col], unit='s')
    return df



In [14]:
# hide
# tests 

df = get_alarm_log('london.my-netdata.io')

expected_cols = ['hostname', 'unique_id', 'alarm_id', 'alarm_event_id', 'name', 'chart', 'family', 'processed', 'updated', 'exec_run', 'exec_failed', 'exec', 'recipient', 'exec_code', 'source', 'units', 'when', 'duration', 'non_clear_duration', 'status', 'old_status', 'delay', 'delay_up_to_timestamp', 'updated_by_id', 'updates_id', 'value_string', 'old_value_string', 'last_repeat', 'silenced', 'info', 'value', 'old_value', 'no_clear_notification']

assert list(df.columns) == expected_cols
assert len(df) >= 1

In [15]:
# hide
# tests

# test grouping behaves as expected

# get some test data
test_host = 'london.my-netdata.io'
test_charts = ['system.load']

# get raw data
df_last100 = get_data(test_host, test_charts, after=-100, before=0, col_sep='|')

# direclty get aggregations
df_avg = get_data(test_host, test_charts, after=-100, before=0, col_sep='|', points=1, group='average')
df_std = get_data(test_host, test_charts, after=-100, before=0, col_sep='|', points=1, group='stddev')
df_min = get_data(test_host, test_charts, after=-100, before=0, col_sep='|', points=1, group='min')
df_max = get_data(test_host, test_charts, after=-100, before=0, col_sep='|', points=1, group='max')

# calc by hand
df_last100_avg = df_last100.mean()
df_last100_std = df_last100.std()
df_last100_min = df_last100.min()
df_last100_max = df_last100.max()

# get diffs
avg_diffs = round(abs(df_avg - df_last100_avg), 2)
std_diffs = round(abs(df_std - df_last100_std), 2)
min_diffs = round(abs(df_min - df_last100_min), 2)
max_diffs = round(abs(df_max - df_last100_max), 2)

# assert abs differences are small
tolerance = 0.2
assert (avg_diffs <= tolerance).values.tolist() == [[True, True, True]]
assert (std_diffs <= tolerance).values.tolist() == [[True, True, True]]
assert (min_diffs <= tolerance).values.tolist() == [[True, True, True]]
assert (max_diffs <= tolerance).values.tolist() == [[True, True, True]]

In [16]:
# export


def get_allmetrics(host, charts: list = None, wide: bool = False, col_sep: str = '|', sort_cols: bool = True,
                   user: str = None, pwd: str = None, protocol: str = 'http') -> pd.DataFrame:
    """Get allmetrics into a df.  
    
    ##### Parameters:  
    - **host** `str` The host we want to get the alarm log from.
    - **charts** `list` A list of charts to pull data for.
    - **wide** `bool` True if you want to return the data in wide format as opposed to long.
    - **user** `str` A username to use if netdata is password protected.
    - **pwd** `str` A password to use if netdata is password protected.
    - **protocol** `str` 'http' or 'https'.
    
    ##### Returns:  
    - **df** `pd.DataFrame` A df of the latest data from allmetrics.
    
    """
    
    url = f'{protocol}://{host}/api/v1/allmetrics?format=json'
    if user and pwd:
        raw_data = requests.get(url, auth=HTTPBasicAuth(user, pwd)).json()
    else:
        raw_data = requests.get(url).json()
    if charts is None:
        charts = list(raw_data.keys())
    data = []
    for k in raw_data:
        if k in charts:
            time = raw_data[k]['last_updated']
            dimensions = raw_data[k]['dimensions']
            for dimension in dimensions:
                # [time, chart, name, value]
                data.append(
                    [time, k, "{}{}{}".format(k, col_sep, dimensions[dimension]['name']), dimensions[dimension]['value']]
                )
    df = pd.DataFrame(data, columns=['time','chart','dimension','value'])
    if wide:
        df = df[['dimension', 'value']].groupby('dimension').mean().reset_index().pivot_table(columns=['dimension'])
        if sort_cols:
            df = df.reindex(sorted(df.columns), axis=1)
    return df



In [17]:
# hide
# tests

host = 'london.my-netdata.io'
df = get_allmetrics(host)

assert len(df) >= 2000
assert list(df.columns) == ['time','chart','dimension','value']
assert 'system.cpu' in list(df.chart.unique())